# Setup Lengkap

In [ ]:
 # @title 🧩 Install Library dan Clone Repo
# @markdown 👈 Klik untuk menginstal semua dependensi

import sys
import os

# ✅ Fungsi log sederhana
def log(msg, icon="📌"):
    print(f"{icon} {msg}")

# 📥 Clone repo GitHub jika belum ada
repo_dir = "/content/video_toolkit"
if not os.path.exists(repo_dir):
    !git clone https://github.com/lIlSkaSkaSkalIl/Video-to-Teks.git {repo_dir}
else:
    log("Repository video_toolkit sudah tersedia.", icon="📁")

# ➕ Tambahkan ke sys.path
if repo_dir not in sys.path:
    sys.path.append(repo_dir)
    log("Menambahkan repo ke sys.path untuk import modul.", icon="➕")

log("Memulai instalasi dependensi...", icon="📦")

# 🎞️ Sistem & media processing
log("Mengupdate sistem & menginstal ffmpeg + aria2...", icon="🧰")
!sudo apt-get update -y > /dev/null
!sudo apt-get install -y ffmpeg aria2 > /dev/null

# 📦 Python libraries
log("Menginstal Python packages: gdown, yt-dlp, tqdm, ffmpeg-python...", icon="🐍")
!pip install -q --upgrade gdown yt-dlp tqdm ffmpeg-python

log("Menginstal Pyrogram dan TgCrypto...", icon="🔐")
!pip install -q --upgrade pyrogram tgcrypto

# 🧠 Whisper (OpenAI)
log("Menginstal OpenAI Whisper...", icon="🧠")
!pip install -q git+https://github.com/openai/whisper.git

log("✅ Semua dependensi berhasil diinstal dan repo siap digunakan.", icon="✅")

In [ ]:
# @title 👣 Buat struktur folder

import os
from utils.messages import log, folder_structure_status  # ⬅️ import fungsi status

# 📂 Lokasi direktori proyek
base_dir = "/content/transkrip_project"
video_dir = os.path.join(base_dir, "video")
audio_dir = os.path.join(base_dir, "audio")
output_dir = os.path.join(base_dir, "output")
log_dir = os.path.join(base_dir, "logs")

# 🏗️ Buat semua folder
os.makedirs(video_dir, exist_ok=True)
os.makedirs(audio_dir, exist_ok=True)
os.makedirs(output_dir, exist_ok=True)
os.makedirs(log_dir, exist_ok=True)

log("Struktur folder berhasil dibuat", icon="📁")

# 🧾 Tampilkan struktur path hasil
folder_structure_status(video_dir, audio_dir, output_dir, log_dir)

# Download Video

In [ ]:
# @title 🍪 [Manual] Upload Cookies
# @markdown - Upload file cookies (Untuk download Twitter yang membutuhkan auth)

from google.colab import files
from utils.messages import log, show_download_summary  # ✅ Import
import os

log("Silakan upload file cookies.txt (hasil dari Kiwi Browser)...", icon="📤")
uploaded = files.upload()

# 🔍 Temukan file .txt
cookies_file = None
for name in uploaded.keys():
    if name.endswith(".txt"):
        cookies_file = name
        break

# 📝 Rename jika perlu
if cookies_file and os.path.exists(cookies_file):
    if cookies_file != "cookies.txt":
        os.rename(cookies_file, "cookies.txt")
        log(f"File '{cookies_file}' diubah menjadi 'cookies.txt'", icon="✏️")
    else:
        log("File sudah bernama cookies.txt", icon="✅")

    # 📄 Tampilkan contoh isi file
    print("📄 Contoh isi cookies.txt:")
    with open("cookies.txt", "r", encoding="utf-8") as f:
        for i in range(3):
            line = f.readline()
            if line:
                print("   ", line.strip())
else:
    log("❌ File .txt tidak valid ditemukan atau gagal upload.", icon="⚠️")

In [ ]:
# @title 🎞️ Unduh Video Twitter
# @markdown - Masukkan URL tweet (single-video / multi-video)
tweet_url = ""  # @param {type: "string"}

import os, re, json, glob, subprocess, datetime
from tqdm import tqdm
from utils.messages import log, show_download_summary  # ✅ Import

# 📁 Siapkan folder
base_dir = "/content/transkrip_project"
video_dir = os.path.join(base_dir, "video")
output_dir = os.path.join(base_dir, "output")
os.makedirs(video_dir, exist_ok=True)
os.makedirs(output_dir, exist_ok=True)

# 🆔 Ambil Tweet ID
match = re.search(r"/status/(\d+)", tweet_url)
if not match:
    raise ValueError("❌ URL tidak valid: Tidak ditemukan Tweet ID.")
tweet_id = match.group(1)

# 🔍 Simulasi metadata
info = None
ydl_opts = {
    "quiet": True,
    "simulate": True,
    "extract_flat": True,
    "dump_single_json": True,
}

log(f"Mendeteksi metadata tweet... ({tweet_url})", icon="🔍")

try:
    import yt_dlp
    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        info = ydl.extract_info(tweet_url, download=False)
except Exception as e:
    log(f"Gagal mendeteksi metadata: {e}", icon="❌")
    if "authentication" in str(e).lower():
        log("Tweet ini kemungkinan membutuhkan cookies.txt (login).", icon="🔐")

# 💾 Simpan JSON jika tersedia
if info:
    json_path = os.path.join(output_dir, f"deteksi_tweet_{tweet_id}.json")
    with open(json_path, "w", encoding="utf-8") as f:
        json.dump(info, f, ensure_ascii=False, indent=2)
    log(f"Metadata disimpan ke: {json_path}", icon="📄")
else:
    log("Metadata tidak tersedia atau tidak disimpan.", icon="⚠️")

# 🍪 Cek cookies
use_cookies = os.path.exists("cookies.txt")
log("Menggunakan cookies.txt" if use_cookies else "Tidak menggunakan cookies", icon="🔐" if use_cookies else "🔓")

# 📥 Proses download (tqdm)
log("Mulai mengunduh video dari Twitter...\n", icon="📥")
command = ["yt-dlp"]
if use_cookies:
    command += ["--cookies", "cookies.txt"]
command += [
    "-f", "best",
    "-o", f"{video_dir}/%(id)s_video.%(ext)s",
    tweet_url
]

progress_bar = tqdm(total=100, desc="📥 Download", unit="%")
process = subprocess.Popen(command, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True)

for line in process.stdout:
    line = line.strip()
    if "%" in line:
        match = re.search(r'(\d{1,3}\.\d)%', line)
        if match:
            percent = float(match.group(1))
            progress_bar.n = int(percent)
            progress_bar.refresh()
    elif "[download]" in line or "Destination" in line:
        print(line)
process.wait()
progress_bar.n = 100
progress_bar.refresh()
progress_bar.close()

log("Download selesai.", icon="✅")

# 📦 Ringkasan hasil
downloaded = glob.glob(os.path.join(video_dir, "*_video.*"))
show_download_summary(tweet_url, tweet_id, use_cookies, info, downloaded, video_dir)

In [ ]:
# @title 🔗 [A1] Video Downloader (non-twitter)
# @markdown - Nama file boleh kosong

download_type = "auto"  # @param ["auto", "google_drive", "direct", "m3u8"]
video_url = ""  # @param {type:"string"}
file_name = ""  # @param {type:"string"}

from datetime import datetime
import os
from utils.messages import log, show_download_info  # ✅ Import

# 📁 Siapkan direktori penyimpanan
video_dir = "/content/transkrip_project/video"
os.makedirs(video_dir, exist_ok=True)
log("Folder video disiapkan.", icon="📂")

# 🕒 Penentuan nama file
if not file_name.strip():
    timestamp = datetime.now().strftime("video_%Y%m%d_%H%M%S")
    file_name = f"{timestamp}.mp4"
    log("Nama file otomatis dibuat berdasarkan waktu.", icon="🕒")
elif not file_name.endswith(".mp4"):
    file_name += ".mp4"
    log("Ekstensi .mp4 ditambahkan pada nama file.", icon="✏️")

# 📍 Path output final
output_path = os.path.join(video_dir, file_name)

# 🧾 Tampilkan informasi teknis
show_download_info(video_url, download_type, output_path)

In [ ]:
# @title 🚀 [A2] Proses Download Berdasarkan Jenis yang Dipilih
# @markdown 👈 Klik untuk mengunduh

import subprocess
import re
import os
import shutil
from utils.messages import log, download_summary, show_tool_detection

def extract_drive_id(url):
    patterns = [
        r"drive\.google\.com\/file\/d\/([^\/]+)",
        r"drive\.google\.com\/open\?id=([^&]+)",
        r"drive\.google\.com\/uc\?id=([^&]+)",
    ]
    for pattern in patterns:
        match = re.search(pattern, url)
        if match:
            return match.group(1)
    return None

def is_m3u8(url):
    return url.endswith(".m3u8") or ".m3u8?" in url

def is_drive(url):
    return "drive.google.com" in url

# 🔧 Pastikan dependensi (silent)
log("Memastikan dependensi...", icon="🛠️")
!apt -qq install -y ffmpeg aria2 > /dev/null
!pip install -q --upgrade yt-dlp > /dev/null
!pip install -q --user gdown > /dev/null

# 🌐 Deteksi alat
tool = download_type
if tool == "auto":
    if is_drive(video_url):
        tool = "google_drive"
    elif is_m3u8(video_url):
        tool = "m3u8"
    else:
        tool = "direct"

show_tool_detection(tool)

# ===================== Proses Unduh =====================
if tool == "google_drive":
    log("Mengunduh dari Google Drive...", icon="📁")
    import gdown
    drive_id = extract_drive_id(video_url)
    if not drive_id:
        raise ValueError("❌ Tidak dapat mendeteksi ID file dari URL Google Drive.")
    gdown.download(f"https://drive.google.com/uc?id={drive_id}", output_path, quiet=False)

elif tool == "m3u8":
    log("Mengunduh dari M3U8 dengan yt-dlp + aria2c...", icon="📺")

    temp_path = os.path.join(video_dir, "temp_m3u8")
    os.makedirs(temp_path, exist_ok=True)

    cmd = [
        "yt-dlp",
        "--downloader", "aria2c",
        "--downloader-args", "aria2c:-x 16 -j 32 -s 32 -k 1M",
        "-f", "best",
        "-o", os.path.join(temp_path, "%(id)s.%(ext)s"),
        video_url
    ]

    process = subprocess.Popen(cmd, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True)
    for line in process.stdout:
        print(f"\r{line.strip()[:150]}", end="", flush=True)
    process.wait()

    downloaded_files = [f for f in os.listdir(temp_path) if f.endswith((".mp4", ".mkv", ".webm"))]
    if not downloaded_files:
        raise FileNotFoundError("❌ Tidak ada file hasil download di folder sementara.")

    temp_download_path = os.path.join(temp_path, downloaded_files[0])
    shutil.move(temp_download_path, output_path)
    log(f"File berhasil dipindahkan ke: {output_path}", icon="✅")

elif tool == "direct":
    log("Mengunduh dari Direct Link dengan yt-dlp...", icon="📥")
    cmd = [
        "yt-dlp",
        "-f", "best",
        "-o", output_path,
        "--no-warnings",
        "--retries", "3",
        video_url
    ]
    process = subprocess.Popen(cmd, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True)
    for line in process.stdout:
        print(f"\r{line.strip()[:150]}", end="", flush=True)
    process.wait()

else:
    raise ValueError("❌ Jenis download tidak dikenali!")

# ✅ Ringkasan akhir
download_summary(output_path)

# Ekstrak Audio dari Video

In [ ]:
# @title 🔉 Ekstrak Audio
# @markdown - Masukkan file Videonya
video_input_path = ""  # @param {type:"string"}

import os
from utils.messages import log, audio_success  # ✅ Impor fungsi status

# 🎯 Siapkan folder output
audio_dir = "/content/transkrip_project/audio"
os.makedirs(audio_dir, exist_ok=True)

# 🎬 Ambil nama video & tentukan path output audio
video_name = os.path.splitext(os.path.basename(video_input_path))[0]
audio_path = os.path.join(audio_dir, f"{video_name}.wav")

# 📦 Konversi ukuran file
def human_readable_size(size_bytes):
    for unit in ['B', 'KB', 'MB', 'GB', 'TB']:
        if size_bytes < 1024:
            return f"{size_bytes:.2f} {unit}"
        size_bytes /= 1024
    return f"{size_bytes:.2f} PB"

# ♻️ Hapus jika sudah ada
if os.path.exists(audio_path):
    os.remove(audio_path)
    log("File audio lama ditemukan dan dihapus", icon="♻️")

# 🔊 Eksekusi ekstraksi
log("Menjalankan ekstraksi audio...", icon="🔊")
print(f"🎬 Input Video  : {video_input_path}")
print(f"🎧 Output Audio : {audio_path}")

!ffmpeg -i "{video_input_path}" -vn -acodec pcm_s16le -ar 16000 -ac 1 "{audio_path}" -y -loglevel error

# ✅ Konfirmasi hasil
if os.path.exists(audio_path):
    size_str = human_readable_size(os.path.getsize(audio_path))
    log("Audio berhasil diekstrak!", icon="✅")
    audio_success(audio_path, size_str)
else:
    log("Gagal mengekstrak audio.", icon="❌")

# Audio to Teks

In [ ]:
# @title 🧠 [1] Pilih Model & Transkripsi Audio
# @markdown - Hasil disimpan otomatis ke `/content/transkrip_project/output`

# 🔘 Pilih model
model_choice = "base"  # @param ["tiny", "base", "small", "medium", "large"]
audio_input_path = ""  # @param {type:"string"}

import whisper
import time
import os
import json
from utils.messages import log  # ✅ Import log dari messages

# 🔧 Folder output
output_dir = "/content/transkrip_project/output"
os.makedirs(output_dir, exist_ok=True)

# 🧪 Validasi input
if not os.path.exists(audio_input_path):
    raise FileNotFoundError(f"❌ File tidak ditemukan: {audio_input_path}")

# 🧠 Muat model & mulai transkripsi
log(f"Memuat model Whisper: {model_choice}", "🧠")
start = time.time()

model = whisper.load_model(model_choice)
result = model.transcribe(audio_input_path, verbose=True)

end = time.time()
log(f"Transkripsi selesai dalam {end - start:.2f} detik", "⏱️")

# 💾 Simpan hasil
json_output_path = os.path.join(output_dir, "segments.json")
with open(json_output_path, "w", encoding="utf-8") as f:
    json.dump(result["segments"], f, ensure_ascii=False, indent=2)

# 📋 Ringkasan hasil
log(f"Hasil transkripsi disimpan ke: {json_output_path}", "✅")
log("Cuplikan teks:", "📝")
print(result["text"][:100] + "...")

In [ ]:
# @title 📝 [2] Ekspor ke Subtitle (.srt) dengan Auto-Split Teks
# @markdown - Atur maksimal jumlah karakter per baris subtitle
json_path = ""  # @param {type:"string"}
max_chars_per_line = 80  # @param {type:"slider", min:40, max:120, step:5}

import os
import json
import textwrap
from datetime import timedelta
from utils.messages import log  # ✅ Import log dari file messages

# 🕒 Konversi detik ke format waktu SRT
def format_time(seconds):
    t = timedelta(seconds=seconds)
    full = str(t).split(".")[0].zfill(8)
    ms = f"{int(t.microseconds/1000):03d}"
    return f"{full},{ms}"

# 📚 Bungkus teks jika terlalu panjang
def wrap_text(text, width=80):
    return "\n".join(textwrap.wrap(text, width=width))

# 📥 Baca file JSON hasil transkripsi
if not os.path.exists(json_path):
    raise FileNotFoundError(f"❌ File tidak ditemukan: {json_path}")

with open(json_path, "r", encoding="utf-8") as f:
    segments = json.load(f)

log(f"Membaca transkrip dari: {json_path}", "📄")
log(f"Jumlah segmen: {len(segments)}", "🔢")

# 🧾 Bangun isi subtitle
srt_lines = []
for i, seg in enumerate(segments, 1):
    start = format_time(seg["start"])
    end = format_time(seg["end"])
    text = wrap_text(seg["text"].strip(), width=max_chars_per_line)

    srt_lines.append(f"{i}")
    srt_lines.append(f"{start} --> {end}")
    srt_lines.append(text)
    srt_lines.append("")

# 💾 Simpan sebagai .srt
json_base = os.path.splitext(os.path.basename(json_path))[0]
srt_output_path = os.path.join("/content/transkrip_project/output", f"{json_base}.srt")

with open(srt_output_path, "w", encoding="utf-8") as f:
    f.write("\n".join(srt_lines))

log("Subtitle berhasil disimpan ke:", "✅")
print(f"📍 {srt_output_path}")

# Upload to Gogle Drive

In [9]:
# @title 🔌 [Mount] Sambungkan Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# @title ☁️ Upload ke Google Drive (Manual)

drive_target_folder = ""  # @param {type:"string"}
drive_filename = ""  # @param {type:"string"}
file_to_upload = ""  # @param {type:"string"}

import os, shutil, sys, datetime
from utils.messages import log  # ✅ Import log dari file messages

# 🔐 Cek apakah Google Drive sudah di-mount
if not os.path.exists("/content/drive"):
    log("Google Drive belum di-mount.", "❌")
    log("Jalankan dulu:", "🔧")
    print("  from google.colab import drive\n  drive.mount('/content/drive')")
    sys.exit()

# 📂 Validasi file sumber
if not os.path.exists(file_to_upload):
    log(f"File tidak ditemukan: {file_to_upload}", "❌")
    sys.exit()

# 📄 Info file sumber
file_size_mb = os.path.getsize(file_to_upload) / (1024 * 1024)
file_name_local = os.path.basename(file_to_upload)
log(f"File sumber ditemukan:", "🔍")
print(f"   📄 Nama     : {file_name_local}")
print(f"   📦 Ukuran   : {file_size_mb:.2f} MB")
print(f"   📂 Lokasi   : {file_to_upload}")

# 📝 Gunakan nama asli jika tidak diisi
if not drive_filename:
    drive_filename = file_name_local

# 📍 Tentukan path tujuan
drive_target_path = os.path.join(drive_target_folder, drive_filename)

# 🛡️ Validasi path tujuan
if not drive_target_path.startswith("/content/drive/"):
    log("Path tujuan harus berada di dalam `/content/drive/`", "❌")
    sys.exit()

# 📁 Buat folder jika belum ada
if not os.path.exists(drive_target_folder):
    os.makedirs(drive_target_folder)
    log(f"Folder baru dibuat: {drive_target_folder}", "📁")
else:
    log(f"Folder tujuan tersedia: {drive_target_folder}", "📁")

# ☁️ Mulai unggah
log(f"Mengunggah file ke Google Drive...", "📤")
shutil.copy(file_to_upload, drive_target_path)

# 🕒 Waktu unggah selesai
upload_time = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")

# ✅ Ringkasan akhir
log(f"File berhasil dikirim ke Google Drive!", "✅")
print(f"   📄 Nama File     : {drive_filename}")
print(f"   📦 Ukuran        : {file_size_mb:.2f} MB")
print(f"   📂 Lokasi GDrive : {drive_target_path}")
print(f"   🕒 Selesai pada   : {upload_time}")

In [ ]:
# @title ☁️ Upload Banyak File ke Google Drive (Auto Rename)
# @markdown - `upload_source_folder`: Folder lokal berisi file yang akan diupload

# @markdown - `drive_target_folder`: Tujuan upload di Google Drive

upload_source_folder = ""  # @param {type:"string"}
drive_target_folder = ""  # @param {type:"string"}

import os, shutil, sys, datetime
from utils.messages import log  # ✅ Import log dari file messages

# 🔐 Cek Google Drive
if not os.path.exists("/content/drive"):
    log("Google Drive belum di-mount.", "❌")
    log("Jalankan dulu:", "🔧")
    print("  from google.colab import drive\n  drive.mount('/content/drive')")
    sys.exit()

# 📁 Validasi folder sumber
if not os.path.exists(upload_source_folder):
    log(f"Folder sumber tidak ditemukan: {upload_source_folder}", "❌")
    sys.exit()

# 📄 Ambil file saja, bukan folder
file_list = [f for f in os.listdir(upload_source_folder)
             if os.path.isfile(os.path.join(upload_source_folder, f))]

if not file_list:
    log("Tidak ada file yang ditemukan di folder tersebut.", "⚠️")
    sys.exit()

# 📂 Cek/buat folder tujuan
if not os.path.exists(drive_target_folder):
    os.makedirs(drive_target_folder)
    log(f"Folder tujuan baru dibuat: {drive_target_folder}", "📁")
else:
    log(f"Folder tujuan tersedia: {drive_target_folder}", "📁")

# 🚀 Mulai proses upload
log(f"Mengunggah {len(file_list)} file ke Google Drive...\n", "📤")

for i, filename in enumerate(file_list, 1):
    local_path = os.path.join(upload_source_folder, filename)

    # 🔁 Auto Rename jika nama file sudah ada
    name_only, ext = os.path.splitext(filename)
    drive_path = os.path.join(drive_target_folder, filename)
    counter = 1
    while os.path.exists(drive_path):
        new_name = f"{name_only}_{counter}{ext}"
        drive_path = os.path.join(drive_target_folder, new_name)
        counter += 1
    final_name = os.path.basename(drive_path)

    # 🔍 Info ukuran
    file_size_mb = os.path.getsize(local_path) / (1024 * 1024)

    # 📥 Salin file
    shutil.copy(local_path, drive_path)
    upload_time = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")

    # 📌 Ringkasan per file
    if counter > 1:
        log(f"Nama file sudah ada, diubah menjadi: {final_name}", "⚠️")
    log(f"({i}/{len(file_list)}) {final_name}", "✅")
    print(f"   📦 Ukuran   : {file_size_mb:.2f} MB")
    print(f"   📂 Tujuan   : {drive_path}")
    print(f"   🕒 Waktu    : {upload_time}\n")

log("Semua file berhasil diupload (dengan rename otomatis jika perlu).", "🎉")

# Upload to Telegram

In [ ]:
# @title 🎯 Ambil Thumbnail dan Durasi Video
video_path = ""  # @param {type:"string"}

import os
import subprocess
import json

# 📁 Buat folder metadata dan thumbnail
meta_dir = "/content/transkrip_project/meta"
thumb_dir = "/content/transkrip_project/thumbnails"
os.makedirs(meta_dir, exist_ok=True)
os.makedirs(thumb_dir, exist_ok=True)

# 📝 Nama file dasar
basename = os.path.splitext(os.path.basename(video_path))[0]
thumbnail_path = os.path.join(thumb_dir, f"{basename}_thumb.jpg")
json_path = os.path.join(meta_dir, f"{basename}_meta.json")

# 📸 Ambil thumbnail dari detik ke-1
print("📸 Mengambil thumbnail...")
subprocess.run([
    "ffmpeg", "-y",
    "-i", video_path,
    "-ss", "00:00:01.000",
    "-vframes", "1",
    "-q:v", "2",
    thumbnail_path
], stdout=subprocess.PIPE, stderr=subprocess.PIPE)

# ⏱️ Ambil durasi, format, dan ukuran file
def extract_video_info(path):
    result = subprocess.run(
        ["ffprobe", "-v", "error", "-select_streams", "v:0",
         "-show_entries", "format=duration,format_name,size", "-of", "json", path],
        stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True
    )
    data = json.loads(result.stdout)
    durasi = float(data['format']['duration'])
    format_name = data['format']['format_name']
    size_bytes = int(data['format']['size'])
    return durasi, format_name, size_bytes

duration_sec, format_name, size_bytes = extract_video_info(video_path)
duration_str = f"{int(duration_sec // 60)}:{int(duration_sec % 60):02d}"

# 💾 Simpan metadata
metadata = {
    "video_path": video_path,
    "thumbnail_path": thumbnail_path,
    "duration": int(duration_sec),
    "duration_str": duration_str,
    "filename": os.path.basename(video_path),
    "format": format_name,
    "size_mb": round(size_bytes / (1024 * 1024), 2)
}
with open(json_path, "w") as f:
    json.dump(metadata, f, indent=2)

# 📊 Tampilkan ringkasan
print("\n📦 Metadata Disimpan:")
print(f"📝 JSON       : {json_path}")
print(f"🎞️ Video Asli : {metadata['filename']}")
print(f"🖼️ Thumbnail  : {os.path.basename(thumbnail_path)}")
print(f"⏱️ Durasi     : {duration_str}")
print(f"💾 Ukuran     : {metadata['size_mb']} MB")
print(f"📀 Format     : {format_name.upper()}")

In [ ]:
# @title ✈️ Batch Upload Video ke Bot Telegram

meta_dir = "/content/transkrip_project/meta"
CHAT_ID = 123      # @param {type:"number"}
API_ID = 123         # @param {type:"number"}
API_HASH = ""   # @param {type:"string"}
BOT_TOKEN = ""  # @param {type:"string"}

from pyrogram import Client
from pyrogram.enums import ParseMode
from tqdm import tqdm
import asyncio, json, os, glob, traceback, re, time

# Escape karakter Markdown
def escape_md(text):
    return re.sub(r'([_*\[\]()~`>#+\-=|{}.!])', r'\\\1', text)

# Progress bar di terminal
def progress(current, total):
    bar.update(current - bar.n)

# Status awal upload
def status_awal(filename, filesize_mb, duration, current_index, total_count):
    return f"""🚀 Upload ({current_index}/{total_count})

╭───────── Detail Upload ─────────╮

  📁 Nama   : {filename}
  📦 Ukuran : {filesize_mb:.2f} MB
  🕒 Durasi : {duration} detik
  ⏳ Status : Mengunggah...

╰───────────────────────────╯
"""

# Status sukses (termasuk waktu upload per file)
def status_sukses(filename, current_index, total_count, waktu_upload):
    return f"""✅ Upload Berhasil! ({current_index}/{total_count})

╭───────── Detail Upload ─────────╮

  🎬 File    : {filename}
  📤 Status  : Sukses
  🧹 Cleanup : File dihapus otomatis
  ⏱️ Waktu   : {waktu_upload:.2f} detik

╰───────────────────────────╯
"""

# Status error
def status_error(filename, error_text, current_index, total_count):
    return f"""❌ Upload Gagal! ({current_index}/{total_count})

╭───────── Detail Upload ─────────╮

  📁 File   : {filename}
  ⚠️ Error  : {error_text}

╰───────────────────────────╯
"""

# Kirim video per file
async def kirim_video(app, meta_path, current_index, total_count):
    try:
        start_upload = time.time()

        with open(meta_path, "r") as f:
            meta = json.load(f)

        video_path = meta["video_path"]
        thumbnail_path = meta["thumbnail_path"]
        filename = escape_md(meta["filename"])
        duration = meta["duration"]
        filesize = os.path.getsize(video_path)
        filesize_mb = filesize / (1024 * 1024)

        print(f"\n📤 Mengunggah video {current_index} dari {total_count}: {filename}")
        await app.send_message(
            chat_id=CHAT_ID,
            text=status_awal(filename, filesize_mb, duration, current_index, total_count),
            parse_mode=ParseMode.MARKDOWN
        )

        global bar
        bar = tqdm(total=filesize, unit='B', unit_scale=True, desc=f"📤 Upload {current_index}/{total_count}", dynamic_ncols=True)

        await app.send_video(
            chat_id=CHAT_ID,
            video=video_path,
            thumb=thumbnail_path,
            caption=filename,
            duration=duration,
            supports_streaming=True,
            progress=progress
        )
        bar.close()
        print("✅ Upload berhasil!")

        waktu_upload = round(time.time() - start_upload, 2)
        await app.send_message(
            chat_id=CHAT_ID,
            text=status_sukses(filename, current_index, total_count, waktu_upload),
            parse_mode=ParseMode.MARKDOWN
        )

        for f in [thumbnail_path, meta_path, video_path]:
            if os.path.exists(f):
                os.remove(f)
                print(f"🗑️ Dihapus: {f}")

    except Exception as e:
        error_msg = str(e) or traceback.format_exc()
        print(f"❌ Error saat upload video {current_index} dari {total_count}: {error_msg}")
        await app.send_message(
            chat_id=CHAT_ID,
            text=status_error(
                escape_md(meta.get("filename", os.path.basename(meta_path))),
                escape_md(error_msg),
                current_index,
                total_count
            ),
            parse_mode=ParseMode.MARKDOWN
        )

# Proses batch upload
async def batch_upload():
    async with Client("upload_bot", api_id=API_ID, api_hash=API_HASH, bot_token=BOT_TOKEN) as app:
        meta_files = sorted(glob.glob(os.path.join(meta_dir, "*_video_meta.json")))
        total = len(meta_files)
        print(f"📦 Menemukan {total} file untuk diunggah.")

        if total == 0:
            await app.send_message(
                chat_id=CHAT_ID,
                text="⚠️ Tidak ada file video yang ditemukan untuk diupload.",
                parse_mode=ParseMode.MARKDOWN
            )
            return

        start_time = time.time()
        total_size_bytes = 0

        for meta_path in meta_files:
            try:
                with open(meta_path, "r") as f:
                    meta = json.load(f)
                    total_size_bytes += os.path.getsize(meta["video_path"])
            except:
                pass

        for idx, meta_path in enumerate(meta_files, start=1):
            await kirim_video(app, meta_path, idx, total)
            await asyncio.sleep(2)

        elapsed = time.time() - start_time
        minutes, seconds = divmod(int(elapsed), 60)
        total_size_mb = total_size_bytes / (1024 * 1024)

        await app.send_message(
            chat_id=CHAT_ID,
            text=f"""✅ Batch Upload Selesai!!

📁 Total File   : {total} video
📦 Total Ukuran : {total_size_mb:.2f} MB
⏱️ Total Waktu  : {minutes} menit {seconds} detik

🎉 Semua video berhasil diupload!
""",
            parse_mode=ParseMode.MARKDOWN
        )

# Jalankan
await batch_upload()

# Utility

In [ ]:

# @title 🎬 Embed Subtitle (.srt) → Output MKV
# @markdown - Masukkan path lengkap video dan subtitle (.srt)

video_input_path = ""  # @param {type:"string"}
subtitle_path = ""     # @param {type:"string"}

import os
import subprocess
from utils.messages import log  # ✅ Import log dari file messages

# 📁 Direktori output (pastikan sesuai)
output_dir = "/content/transkrip_project/output"
os.makedirs(output_dir, exist_ok=True)

# ✅ Validasi input
if not os.path.exists(video_input_path):
    log(f"File video tidak ditemukan: {video_input_path}", "❌")
    raise FileNotFoundError()
if not os.path.exists(subtitle_path):
    log(f"File subtitle tidak ditemukan: {subtitle_path}", "❌")
    raise FileNotFoundError()

# 🔄 Ambil nama dasar file
video_basename = os.path.splitext(os.path.basename(video_input_path))[0]
output_video_path = os.path.join(output_dir, f"{video_basename}_with_subtitles.mkv")

# 🎞️ Perintah FFMPEG
cmd = [
    "ffmpeg",
    "-i", video_input_path,
    "-i", subtitle_path,
    "-c", "copy",
    "-c:s", "srt",
    "-map", "0",
    "-map", "1",
    output_video_path
]

log("Menyisipkan subtitle ke video (softsub MKV)...", "🎞️")
process = subprocess.run(cmd, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)

# 🔍 Cek hasil
if os.path.exists(output_video_path):
    size = os.path.getsize(output_video_path) / (1024 * 1024)
    log(f"Video akhir disimpan di: {output_video_path}", "✅")
    log(f"Ukuran file: {size:.2f} MB", "📦")
else:
    log("Proses gagal: file tidak ditemukan setelah embed.", "❌")

In [ ]:
# @title 📝 Ekstrak Subtitle (Softsub) dari Video
# @markdown - Ekstrak subtitle ke file .srt jika tersedia dalam video

video_input_path = ""  # @param {type:"string"}

import os
import subprocess
from utils.messages import log  # ✅ Import log dari file messages

# 📁 Output folder
output_dir = "/content/transkrip_project/output"
os.makedirs(output_dir, exist_ok=True)

# ✅ Validasi input
if not os.path.exists(video_input_path):
    log(f"File video tidak ditemukan: {video_input_path}", "❌")
    raise FileNotFoundError()

# 🔍 Cek apakah video punya subtitle stream
log("Mendeteksi subtitle stream di dalam video...", "🔍")
cmd_check = [
    "ffprobe", "-loglevel", "error",
    "-select_streams", "s",  # hanya subtitle stream
    "-show_entries", "stream=index:stream_tags=language",
    "-of", "default=noprint_wrappers=1",
    video_input_path
]

result = subprocess.run(cmd_check, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
subtitle_info = result.stdout.strip()

if not subtitle_info:
    log("Tidak ditemukan subtitle di dalam video.", "⚠️")
else:
    log("Subtitle ditemukan! Mengekstrak ke .srt...", "📤")

    # Ambil nama dasar file
    base_name = os.path.splitext(os.path.basename(video_input_path))[0]
    subtitle_output = os.path.join(output_dir, f"{base_name}_extracted.srt")

    # Perintah ekstraksi
    cmd_extract = [
        "ffmpeg", "-y", "-i", video_input_path,
        "-map", "0:s:0",  # ambil subtitle pertama
        subtitle_output
    ]
    subprocess.run(cmd_extract, stdout=subprocess.PIPE, stderr=subprocess.PIPE)

    # Cek hasil
    if os.path.exists(subtitle_output):
        size_kb = os.path.getsize(subtitle_output) / 1024
        log(f"✅ Subtitle berhasil diekstrak: {subtitle_output}", "✅")
        log(f"📦 Ukuran file: {size_kb:.2f} KB", "📁")
    else:
        log("❌ Ekstraksi gagal: file .srt tidak ditemukan.", "❌")

In [ ]:
# @title 🧹 [F] Bersihkan File Setelah Proses
# @markdown Pilih folder yang ingin dibersihkan:
hapus_video = True  # @param {type:"boolean"}
hapus_audio = True  # @param {type:"boolean"}
hapus_output = True  # @param {type:"boolean"}
hapus_logs = True  # @param {type:"boolean"}

import os

def log(msg, icon="🔸"):
    print(f"{icon} {msg}")

def delete_all_files(folder, label):
    if not os.path.exists(folder):
        log(f"Folder tidak ditemukan: {folder}", "❌")
        return

    file_list = [f for f in os.listdir(folder) if os.path.isfile(os.path.join(folder, f))]
    if not file_list:
        log(f"Tidak ada file di folder {label}.", "ℹ️")
        return

    log(f"Membersihkan folder: {label}", "🧹")
    for filename in file_list:
        path = os.path.join(folder, filename)
        try:
            os.remove(path)
            log(f"🗑️  {filename} — Dihapus")
        except Exception as e:
            log(f"⚠️  Gagal hapus {filename}: {e}")

# 🧹 Jalankan pembersihan sesuai pilihan
if hapus_video:
    delete_all_files(video_dir, "📁 Video")

if hapus_audio:
    delete_all_files(audio_dir, "🎧 Audio")

if hapus_output:
    delete_all_files(output_dir, "📄 Output")

if hapus_logs:
    delete_all_files(log_dir, "📜 Logs")